#Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)

# Sharookh Ali

## Introduction: A description of the problem and a discussion of the background. (15 marks)

<ol><li>In Module 3, we explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. <li>One interesting idea would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are. 
<li> In this assignment we will compare the  neighborhoods of the two indian cities namely Hyderabad and Mumbai
<li>Is Hyderabad more like Toronto or Mumbai or some other multicultural city? 
<li>In Hyderabad, if someone is looking to open a restaurant, where would we recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office? and other important points will be discussed

# Extracting the data

The data extracted may include:
<ol><li>List of Cafes and foodstalls of Hyderabad with their lattitude and longitude 
<li>List of Hotels, Book stores,Restaurant, Bakaries, and all relavant locations
<li> All data is limited with in 10km radius 

In [1]:
import json 
import requests 
from pandas.io.json import json_normalize 
import numpy as np 
import time
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim 
import folium 
print('Libraries imported.')

Libraries imported.


## Extracting the geograpical coordinates of Hyderabad through geolocator

In [2]:
address = 'Hyderabad'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of' ,address,'are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Hyderabad are 17.360589, 78.4740613.


/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [3]:
neighborhood_latitude=latitude
neighborhood_longitude=longitude

## Using Foursquare credentials to analyse the neighbourhood in Hyderabad

In [4]:
CLIENT_ID = '4PDNWKYQ3G5K3PEKC00MO2CUQWZGZJRTW1BTP5RE1M42VX2I' 
CLIENT_SECRET = '1OL5BDNLNVT0YCQGDKZC5GBUDK5PVMREH4G4PIUAZPDLT0DW' # 
VERSION = '20180604' 

In [5]:
radius = 10000
limit = 200 
# importing the data with URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius,
    limit)
results = requests.get(url).json()

## Determining the catogories of each location in data

In [6]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Showing the list of venues

In [7]:
venues = results['response']['groups'][0]['items']
    
HYD_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
HYD_venues =HYD_venues.loc[:, filtered_columns]
# filter the category for each row
HYD_venues['venue.categories'] = HYD_venues.apply(get_category_type, axis=1)
# clean columns
HYD_venues.columns = [col.split(".")[-1] for col in HYD_venues.columns]
HYD_venues.head(100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Chowmahala Palace,History Museum,17.359300,78.471617
1,Shadaab,Diner,17.368661,78.475572
2,Hotel Shah Ghouse,Snack Place,17.347899,78.470986
3,Pista house,Indian Restaurant,17.356609,78.473637
4,Taj Falaknuma Palace,Resort,17.330118,78.467460
5,Charminar,Monument / Landmark,17.361555,78.474677
6,Laxman Ki Bandi,South Indian Restaurant,17.378895,78.463973
7,Pragati,South Indian Restaurant,17.388088,78.481134
8,Mayur Pan Shop,Juice Bar,17.388894,78.480578
9,Ram ki Bandi,Food Truck,17.383416,78.475502


In [8]:
HYD_venues.shape

(100, 4)

## Grouping them with repect to their Categoty

In [9]:
HYD_venues.groupby(['categories'])
HYD_venues

,name,categories,lat,lng
0,Chowmahala Palace,History Museum,17.359300,78.471617
1,Shadaab,Diner,17.368661,78.475572
2,Hotel Shah Ghouse,Snack Place,17.347899,78.470986
3,Pista house,Indian Restaurant,17.356609,78.473637
4,Taj Falaknuma Palace,Resort,17.330118,78.467460
5,Charminar,Monument / Landmark,17.361555,78.474677
6,Laxman Ki Bandi,South Indian Restaurant,17.378895,78.463973
7,Pragati,South Indian Restaurant,17.388088,78.481134
8,Mayur Pan Shop,Juice Bar,17.388894,78.480578
9,Ram ki Bandi,Food Truck,17.383416,78.475502


## Creating Hyderabad map with Folium

In [10]:
MAP_HYD = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, label in zip(HYD_venues['lat'], HYD_venues['lng'], HYD_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7,
    ).add_to(MAP_HYD)  
    
MAP_HYD